# POI Visit Counts

### real world data

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
import json

def get_agent_df(agent_file,column_name='SIM_VISIT_COUNTS'):
    agent_df = pd.read_csv(agent_file)
    poi_list = []
    for idx,row in agent_df.iterrows():
        schedule = json.loads(row['schedule'])
        for item in schedule:
            desire = item['desire']
            if desire != 'home':
                destination = item['destination']
                poi = (destination['name'],destination['coordinates'][0],destination['coordinates'][1])
                poi_list.append(poi)

    poi_counter = Counter(poi_list)
    poi_counts = []
    for (name, lat, lng), count in poi_counter.items():
        poi_counts.append({
            'LOCATION_NAME': name,
            'LATITUDE': float(lat),
            'LONGITUDE': float(lng),
            column_name: count
        })
    poi_counts_df = pd.DataFrame(poi_counts)
    return poi_counts_df


poi_file = 'data/geo/safegraph-cambridge-poi.csv'
sim_noref_file = 'agents/agents_cambridge_noref.csv'
sim_ref_file = 'agents/agents_cambridge_ref.csv'

poi_df = pd.read_csv(poi_file)
poi_df = poi_df[['LOCATION_NAME','LATITUDE', 'LONGITUDE','RAW_VISIT_COUNTS']]
sim_df_noref = get_agent_df(sim_noref_file,column_name='SIM_VISIT_COUNTS_1')
sim_df_ref = get_agent_df(sim_ref_file,column_name='SIM_VISIT_COUNTS_2')

merged_df = pd.merge(poi_df,sim_df_noref,on=['LOCATION_NAME','LATITUDE', 'LONGITUDE'],how='outer')

merged_df = pd.merge(merged_df,sim_df_ref,on=['LOCATION_NAME','LATITUDE', 'LONGITUDE'],how='outer')

# add mini_counts for kld calculation
min_counts = 10
merged_df = merged_df.fillna(0)
merged_df['RAW_VISIT_COUNTS']  = merged_df['RAW_VISIT_COUNTS'] + min_counts
merged_df['SIM_VISIT_COUNTS_1']  = merged_df['SIM_VISIT_COUNTS_1'] + min_counts
merged_df['SIM_VISIT_COUNTS_2']  = merged_df['SIM_VISIT_COUNTS_2'] + min_counts

merged_df.head(2)

In [ ]:
from scipy.stats import entropy

P = merged_df['RAW_VISIT_COUNTS']/merged_df['RAW_VISIT_COUNTS'].max()
Q1 = merged_df['SIM_VISIT_COUNTS_1']/merged_df['SIM_VISIT_COUNTS_1'].max()
Q2 = merged_df['SIM_VISIT_COUNTS_2']/merged_df['SIM_VISIT_COUNTS_2'].max()

kld_q1 = entropy(P,Q1)
kld_q2 = entropy(P,Q2)

print(kld_q1,kld_q2)

In [ ]:
import pydeck as pdk
import numpy as np

def visualize_poi(df, column_name='RAW_VISIT_COUNTS',max_threshold=1):
    """
    使用热力图和3D柱状图可视化POI访问数据
    
    参数:
        df: 包含位置数据的DataFrame
        column_name: 用于归一化访问量的列名
        mini_height: 柱状图的最小高度
        radius_pixels: 热力图的半径像素
        max_threshold: 最大阈值用于归一化
        sqrt_norm: 是否使用平方根归一化
        elevation_scale: 高度缩放因子
        
    返回:
        pydeck.Deck对象
    """
    # 排序确保高值点显示在上层
    plot_df = df[['LOCATION_NAME','LATITUDE','LONGITUDE',column_name]].copy()
    plot_df = plot_df.sort_values(column_name, ascending=True)
    min_value = np.sqrt(plot_df[column_name]).min()
    max_value = np.sqrt(plot_df[column_name]).max()*max_threshold

    # 归一化处理
    plot_df['weight'] = (np.sqrt(plot_df[column_name])-min_value)/(max_value-min_value)
    
    # 定义视图状态 - 增加倾斜角度以更好地展示3D效果
    view_state = pdk.ViewState(
        latitude=plot_df['LATITUDE'].mean(),
        longitude=plot_df['LONGITUDE'].mean(),
        zoom=12,
        min_zoom=12,
        max_zoom=12,
        pitch=45,  # 增加倾斜角度以显示3D效果
        bearing=0
    )

    # 1. 创建热力图层
    radius_pixels=120
    heatmap_layer = pdk.Layer(
        'HeatmapLayer',
        data=plot_df,
        get_position=['LONGITUDE', 'LATITUDE'],
        get_weight=column_name,
        opacity=0.6,
        color_range=[
            [255, 245, 240, 50],   # 非常淡的红色 (低密度)
            [254, 224, 210, 100],  # 淡红色
            [252, 146, 114, 150],   # 中等红色
            [251, 106, 74, 200],   # 较深红色
            [222, 45, 38, 250],     # 深红色
            [165, 15, 21, 255]     # 非常深的红色 (高密度)
        ],
        radius_pixels=radius_pixels,
        intensity=0.8,
        threshold=0.05,
        pickable=False
    )

    # 2. 创建3D柱状图层
    mini_height=20
    radius=50
    elevation_scale=2000
    column_layer = pdk.Layer(
        'ColumnLayer',
        data=plot_df,
        get_position=['LONGITUDE', 'LATITUDE'],
        get_elevation=f'{mini_height} + weight*{elevation_scale}',
        radius=radius,  # 柱子的半径
        elevation_scale=1,
        get_fill_color=['255', '0', '0', '50+weight*200'], 
        pickable=True,
        auto_highlight=True,
        extruded=True  # 确保柱子是3D的
    )

    # 创建deck（热力图在下，柱状图在上）
    deck = pdk.Deck(
        layers=[heatmap_layer, column_layer],
        initial_view_state=view_state,
        map_style='light',
        tooltip={
            'html': '<b>地点:</b> {LOCATION_NAME}<br/><b>访问量:</b> {' + column_name + '}',
            'style': {
                'color': 'white'
            }
        },
        width=800,
        height=600,
    )
    return deck

In [ ]:
visualize_poi(merged_df,column_name='RAW_VISIT_COUNTS',max_threshold=0.8)

In [ ]:
visualize_poi(merged_df,column_name='SIM_VISIT_COUNTS_1')

In [ ]:
visualize_poi(merged_df,column_name='SIM_VISIT_COUNTS_2')

# Trip Visualization

In [ ]:
import networkx as nx
from shapely.geometry import LineString, Point
import numpy as np
import geopandas as gpd

trip_file = "data/geo/replica-cambridge-roads.geojson"
trip_df = gpd.read_file(trip_file)

# 创建图
G_from_geojson = nx.Graph()

# 处理每个LineString
for idx, row in trip_df.iterrows():
    if row.geometry and row.geometry.geom_type == 'LineString':
        coords = list(row.geometry.coords)
        edge_attrs = {k: v for k, v in row.items() if k != 'geometry'}
        
        for i in range(len(coords) - 1):
            u = f"{coords[i][0]:.6f},{coords[i][1]:.6f}"
            v = f"{coords[i+1][0]:.6f},{coords[i+1][1]:.6f}"
            
            if u not in G_from_geojson:
                G_from_geojson.add_node(u, x=coords[i][0], y=coords[i][1])
            if v not in G_from_geojson:
                G_from_geojson.add_node(v, x=coords[i+1][0], y=coords[i+1][1])
            
            segment = LineString([coords[i], coords[i+1]])
            length = segment.length
            
            G_from_geojson.add_edge(u, v, 
                                   osmid=edge_attrs.get('osmId'),
                                   length=length,
                                   geometry=segment,
                                   **edge_attrs)

print(f"Created network with {len(G_from_geojson.nodes)} nodes and {len(G_from_geojson.edges)} edges")


In [ ]:
import pandas as pd
import geopandas as gpd
from collections import Counter
import json

def get_shortest_path_edges(G, pt1, pt2):
    """
    查找两点间的最短路径并返回路径中的边
    
    参数:
    G (networkx.Graph): 图
    pt1 (tuple): 起点坐标 (lat, lng)
    pt2 (tuple): 终点坐标 (lat, lng)
    
    返回:
    list: 路径中的边列表 (u, v) 或 None 如果找不到路径
    """
    def find_nearest_node(G, point):
        lat, lng = point
        min_dist = float('inf')
        nearest = None
        
        for node, data in G.nodes(data=True):
            node_x, node_y = data['x'], data['y']
            dist = ((node_x - lng)**2 + (node_y - lat)**2)**0.5
            
            if dist < min_dist:
                min_dist = dist
                nearest = node
        
        return nearest
    
    origin_node = find_nearest_node(G, pt1)
    destination_node = find_nearest_node(G, pt2)
    
    if not origin_node or not destination_node:
        return None
    
    try:
        # 获取最短路径的边
        path = nx.shortest_path(G, origin_node, destination_node, weight='length')
        edges = list(zip(path[:-1], path[1:]))
        return edges
    except Exception as e:
        return None

# 加载agent数据
agent_file = 'agents/agents_cambridge_noref.csv'


def get_trip_df(agent_file):
    agent_df = pd.read_csv(agent_file)
    # 统计边的使用次数
    edge_counter = Counter()

    for idx, row in agent_df.iterrows():
        schedule = json.loads(row['schedule'])
        coords = [item['destination']['coordinates'] for item in schedule]
        coords = [(float(lat), float(lng)) for (lat, lng) in coords]
        
        if len(coords) > 1:
            for i in range(len(coords) - 1):
                pt1 = coords[i]
                pt2 = coords[i+1]
                edges = get_shortest_path_edges(G_from_geojson, pt1, pt2)
                if edges:
                    edge_counter.update(edges)

    # 创建包含边属性和计数的DataFrame
    edge_data = []
    for edge, count in edge_counter.items():
        u, v = edge
        if G_from_geojson.has_edge(u, v):
            edge_attrs = G_from_geojson.get_edge_data(u, v)
            edge_info = {
                'from_node': u,
                'to_node': v,
                'sim_count': count,
                'geometry': edge_attrs.get('geometry'),
                'length': edge_attrs.get('length'),
                'osmid': edge_attrs.get('osmid')
            }
            # 添加其他属性
            for key, value in edge_attrs.items():
                if key not in ['geometry', 'length', 'osmid']:
                    edge_info[key] = value
            edge_data.append(edge_info)

    # 创建GeoDataFrame
    edge_gdf = gpd.GeoDataFrame(edge_data, geometry='geometry')
    edge_gdf = edge_gdf[['osmId','roadName','highway','trip_count','sim_count','geometry']]
    return edge_gdf

## Trip Noref

In [ ]:
trip_noref_file = 'agents/agents_cambridge_noref.csv'
trip_df_noref = get_trip_df(trip_noref_file)
trip_df_noref.head(1)

## Trip Ref

In [ ]:
trip_ref_file = 'agents/agents_cambridge_ref.csv'
trip_df_ref = get_trip_df(trip_ref_file)
trip_df_ref.head(1)

## Visualization

In [ ]:
import geopandas as gpd

trip_df_noref = gpd.read_file('agents/sim_trip_noref.geojson')
trip_df_ref = gpd.read_file('agents/sim_trip_ref.geojson')
trip_df_ref.head(1)

In [ ]:
from scipy.stats import entropy

P1 = trip_df_noref['trip_count']/trip_df_noref['trip_count'].max()
Q1 = trip_df_noref['sim_count']/trip_df_noref['sim_count'].max()

P2 = trip_df_ref['trip_count']/trip_df_ref['trip_count'].max()
Q2 = trip_df_ref['sim_count']/trip_df_ref['sim_count'].max()

kld_q1 = entropy(P1,Q1)
kld_q2 = entropy(P2,Q2)

print(kld_q1,kld_q2)

In [ ]:
import pydeck as pdk
from shapely.geometry import mapping  # Added import for mapping
import numpy as np
import pandas as pd

def visualize_trip_data(gdf, column_name="trip_count", width_scale=20,max_threshold=1):
    """
    Visualize trip data on a map using PyDeck
    
    Parameters:
    gdf (GeoDataFrame): The GeoDataFrame containing trip data
    column_name (str): Column name to use for visualization
    width_scale (int): Scale factor for line width
    color_range (list): Custom color range for visualization
    """
    # Convert the GeoDataFrame to a format suitable for PyDeck
    path_data = []
    
    # Get min and max values for normalization (using square root)
    sqrt_values = gdf[column_name].apply(lambda x: np.sqrt(x))
    min_value = sqrt_values.min()
    max_value = sqrt_values.max()*max_threshold
    
    for idx, row in gdf.iterrows():
        if row.geometry and row.geometry.geom_type == 'LineString':
            # Extract coordinates from the LineString
            coords = list(mapping(row.geometry)['coordinates'])
            
            # Get the value for this path and apply square root
            count_value = row[column_name]
            sqrt_value = np.sqrt(count_value)
            # Normalize the value for scaling (0 to 1)
            normalized_value = (sqrt_value - min_value) / (max_value - min_value)
            # Add path data
            path_data.append({
                'path': coords,
                'width': normalized_value * 5 * width_scale,  # Adjusted width scaling
                'name': row['roadName'] if pd.notna(row['roadName']) else 'Unknown',
                'type': row['highway'] if pd.notna(row['highway']) else 'Unknown',
                'osmId': row['osmId'],
                'count_value': count_value,
                'normalized_value': normalized_value,
                'opacity': normalized_value * 0.8 + 0.2,  # Opacity between 0.2 and 1.0
                'z_index': count_value  # Add z-index based on count_value
            })
    

    color_range = [
            [0, 0, 225, 20],   
            [0, 0, 225, 20],   
            [0, 0, 225, 20],   
            [0, 0, 225, 20],   
            [0, 0, 225, 20],   
            [0, 0, 225, 60],   
            [0, 0, 225, 60],   
            [0, 0, 225, 60],   
            [0, 0, 225, 60],   
            [0, 0, 225, 60],   
        ]
    
    # Create a color scale function
    def get_color(value, color_range):
        value = max(0, min(1, value))  # Clamp between 0 and 1
        if value == 1:
            return color_range[-1]
        num_segments = len(color_range) - 1
        segment = int(value * num_segments)
        segment_value = (value * num_segments) - segment
        color = [
            int(color_range[segment][0] + (color_range[segment+1][0] - color_range[segment][0]) * segment_value),
            int(color_range[segment][1] + (color_range[segment+1][1] - color_range[segment][1]) * segment_value),
            int(color_range[segment][2] + (color_range[segment+1][2] - color_range[segment][2]) * segment_value),
            int(color_range[segment][3])
        ]
        return color
    
    # Apply color to each path
    for path in path_data:
        path['color'] = get_color(path['normalized_value'], color_range)
    
    # 2. Sort path_data by count_value so higher values are drawn last (on top)
    path_data.sort(key=lambda x: x['count_value'])
    
    # Set up the view state
    view_state = pdk.ViewState(
        latitude=gdf.geometry.centroid.y.mean(),
        longitude=gdf.geometry.centroid.x.mean(),
        zoom=12,
        min_zoom=12,
        max_zoom=12,
        pitch=0,
        bearing=0
    )
    
    # Set up the layer with opacity and width based on normalized value
    path_layer = pdk.Layer(
        'PathLayer',
        data=path_data,
        get_path='path',
        get_width='width',
        get_color='color',
        width_min_pixels=1,
        width_max_pixels=100,
        get_opacity='opacity',
        rounded=True,
        pickable=True,
        auto_highlight=True
    )
    
    # Set up tooltip
    tooltip = {
        "html": "<b>{name}</b><br/>Type: {type}<br/>Traffic: {count_value}",
        "style": {
            "backgroundColor": "steelblue",
            "color": "white"
        }
    }
    
    # Create the deck
    deck = pdk.Deck(
        layers=[path_layer],
        initial_view_state=view_state,
        tooltip=tooltip,
        map_style='light', 
        width=800,
        height=600,
    )
    
    return deck

In [ ]:
visualize_trip_data(trip_df_noref,column_name='trip_count', width_scale=16,max_threshold=0.6)

In [ ]:
visualize_trip_data(trip_df_noref,column_name='sim_count',width_scale=25,max_threshold=1)

In [ ]:
visualize_trip_data(trip_df_ref,column_name='sim_count',width_scale=25,max_threshold=1)